In [1]:
import numpy as np
import pandas as pd
import gc
import time
from pickle import dump
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

In [2]:
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay
from imblearn.over_sampling import SMOTE 

In [3]:
# pip install imblearn

In [4]:
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import confusion_matrix

In [5]:
X=pd.read_csv('../input/data_process.csv', low_memory=False)

In [6]:
#X=X.sample(150000)

In [10]:
X.shape

(356251, 799)

In [11]:
X.head()

,Unnamed: 0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,0,0,100002,1.0,0,0,0,0,202500.0,406597.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,100004,0.0,0,1,0,0,67500.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,100006,0.0,1,0,0,0,135000.0,312682.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,4,4,100007,0.0,0,0,0,0,121500.0,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#df.columns.tolist()

# Préprocessing 

In [13]:
X = X[X['TARGET'].notnull()] 

In [14]:
X = X.drop("Unnamed: 0", axis=1)

In [15]:
X = X.drop("index", axis=1)

In [16]:
X = X.drop("SK_ID_CURR", axis=1)

In [17]:
Y = X["TARGET"]

In [18]:
X= X.drop("TARGET", axis=1)

## Gestion des valeurs manquantes

In [19]:
ratio_nas = X.isna().sum()/X.shape[0]

In [20]:
useful_columns_less_than_ratio = ratio_nas[ratio_nas<0.25].index

In [21]:
X = X[useful_columns_less_than_ratio]

In [22]:
X.shape

(307507, 507)

In [23]:
X[~np.isfinite(X)] = np.nan

In [24]:
imputer = SimpleImputer(strategy="median")

In [25]:
columns_keep = X.columns

In [26]:
X = imputer.fit_transform(X)

In [27]:
X = pd.DataFrame(X, columns=columns_keep)

In [28]:
X

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,INSTAL_AMT_INSTALMENT_MEAN,INSTAL_AMT_INSTALMENT_SUM,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_MAX,INSTAL_AMT_PAYMENT_MEAN,INSTAL_AMT_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM,INSTAL_COUNT
0,0.0,0.0,0.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,...,11559.247105,219625.695,9251.775,53093.745,11559.247105,219625.695,-49.0,-315.421053,-5993.0,19.0
1,1.0,0.0,1.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,...,64754.586000,1618864.650,6662.970,560835.360,64754.586000,1618864.650,-544.0,-1385.320000,-34633.0,25.0
2,0.0,1.0,0.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,...,7096.155000,21288.465,5357.250,10573.965,7096.155000,21288.465,-727.0,-761.666667,-2285.0,3.0
3,1.0,0.0,0.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,...,62947.088438,1007153.415,2482.920,691786.890,62947.088438,1007153.415,-12.0,-271.625000,-4346.0,16.0
4,0.0,0.0,0.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,...,12666.444545,835985.340,0.180,22678.785,12214.060227,806127.975,-14.0,-1032.242424,-68128.0,66.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307502,0.0,0.0,1.0,0.0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327.0,...,7492.924286,52450.470,6605.910,12815.010,7492.924286,52450.470,-38.0,-156.285714,-1094.0,7.0
307503,1.0,0.0,0.0,0.0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775.0,...,10069.867500,60419.205,10046.880,10074.465,10069.867500,60419.205,-2327.0,-2393.833333,-14363.0,6.0
307504,1.0,0.0,0.0,0.0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966.0,...,4399.707857,61595.910,27.270,5575.185,4115.915357,57622.815,-1738.0,-2387.428571,-33424.0,14.0
307505,1.0,0.0,0.0,0.0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961.0,...,10239.832895,194556.825,2296.440,19065.825,10239.832895,194556.825,-18.0,-161.263158,-3064.0,19.0


## Gestion des outliers

In [29]:
# from sklearn.ensemble import IsolationForest
# clf = IsolationForest(random_state=0).fit(X)
# clf.predict(X)

In [30]:
# numerics_columns=X.select_dtypes(include = ['int32','float64']).columns

In [31]:
# df_num=df[numerics_columns]

In [32]:
# df_num=df_num.sample(8)

In [33]:
# numeric_value = df_num.select_dtypes(include = ['int32','float64']).columns
# for column in numeric_value:
#     plt.figure(figsize = (10,2))
#     sns.set(font_scale=1)
#     sns.distplot(df_num[column], bins=50)
#     chaine = 'Distribution de : ' + column
#     plt.title(chaine)
#     plt.xlabel(column)
#     plt.show()

### Graphique via boxplot pour l'identification des points abérrants

In [34]:
# sns.set(font_scale=2)
# for column in numeric_value[1:]:
#     plt.figure(figsize = (9,7))
#     sns.boxplot(df_num[column])
#     chaine = 'Distribution de : ' + column
#     plt.title(chaine)
#     plt.xlabel(column)
#     plt.show()
# sns.set(font_scale=1)

In [35]:
# dict_quantile = {} 
# for res in set (X["AMT_INCOME_TOTAL"]): 
#     mask_res = X["AMT_INCOME_TOTAL"]=="25000" 
#     data_res=X.loc[mask_res]
#     if data_res.shape[0]>0: 
#         q1=X[numeric_columns].quantile(q=0.25)
#         q3=X[numeric_columns].quantile(q=0.75)
#         q2=X[numeric_columns].quantile(q=0.5)
#         IQR = q3-q1
#         borne_sup = q3 + 1.5*IQR 
#         borne_inf = q1 - 1.5*IQR 
#         dict_quantile[res]=(borne_inf, borne_sup, q2) 

In [36]:
# def impute_outliers(row: pd.Series):
#     ''' 
#         # METHODE DE DOCSTRING AVEC TOUS LES COMMENTAIRES: return, etc
#        elle permet d'imputer les valeurs abérantes par la moyenne lorsque els données sont 
#        infé aux 1 quartilers et sup aux 3ème quantilles
#        :param row : pandas Series
#        :return the tranformated row
#     '''
#     for name_col_numerique in numeric_columns:
#         category_pro=str(row["AMT_INCOME_TOTAL"])
#         if category_pro in dict_quantile: 
#             valeur_dico=dict_quantile[category_pro] 
#             if row[name_col_numerique]<valeur_dico [0][name_col_numerique] or row [name_col_numerique] > valeur_dico[1][name_col_numerique]:
#                 row [name_col_numerique]=valeur_dico[2][name_col_numerique]
#     return row 

In [37]:
# df2=df2.apply(impute_outliers, axis=1)

## Vérification s'il existe encore des outliers par matrice ou graphique

In [38]:
X.to_csv("X_clean.csv", index=False) # index false permet d'enlever la colonne unammed
Y.to_csv("Y_clean.csv", index=False)